In [ ]:
!python -m pip install --user -r ../requirements.txt

In [ ]:
!python -m pip install --user  opencv-python-headless

In [ ]:
!python -m pip install --user layoutparser # Install the base layoutparser library with
!python -m pip install --user  "layoutparser[layoutmodels]" # Install DL layout model toolkit
!python -m pip install --user "layoutparser[ocr]" # Install OCR toolkit

In [ ]:
!python -m pip install --user 'git+https://github.com/facebookresearch/detectron2.git'

In [ ]:
!sudo apt install --yes libgl1-mesa-glx

In [ ]:
#pip install -U 'git+https://github.com/nikhilweee/iopath'

* restart kernel here

In [ ]:
!pip install "Pillow<=9.5.0"

In [ ]:
import layoutparser as lp
import cv2
import numpy as np
import pdf2image
import os
from PIL import Image

In [ ]:
PATH_OF_IMAGES="./"
TESSERACT_LANGUAGE="fra"
SHOW_IMAGE=True
PRINT_DETECTED_TITLE=False

In [ ]:
model = lp.models.Detectron2LayoutModel(
    config_path ="../config.yaml",
    model_path ="../model_final.pth",
    extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.5],
    label_map={0: "Text", 1: "Title", 2: "List", 3: "Table", 4: "Figure"},
)

In [ ]:
jpg_file="images/A59L18012139-2.jpg"

In [ ]:
img=Image.open(jpg_file)
one_image_np = np.asarray(img)
layout = model.detect(one_image_np)

In [ ]:
image_width, image_height= img.size

In [ ]:
list_of_results=[]
for i,block in enumerate(layout.to_dict()["blocks"]):
    block_dict=dict()
    block_dict["id"]="result"+str(i)
    block_dict["type"]="rectanglelabels"
    block_dict["from_name"]="label"
    block_dict["to_name"]="image"
    block_dict["original_width"]=image_width
    block_dict["original_height"]=image_height
    block_dict["image_rotation"]=0
    values_dict=dict()
    values_dict["rotation"]=0
    values_dict["x"]=block["x_1"]*100/image_width
    values_dict["y"]=block["y_1"]*100/image_height
    width=(abs(block["x_1"]-block["x_2"]))*100/image_width
    height=(abs(block["y_1"]-block["y_2"]))*100/image_height
    values_dict["width"]=width
    values_dict["height"]=height
    values_dict["rectanglelabels"]=[block["type"]]
    block_dict["value"]=values_dict
    list_of_results.append(block_dict)

In [ ]:
dict_one_prediction=dict()
dict_one_prediction["model_version"]="one"
dict_one_prediction["score"]=0.5
dict_one_prediction["result"]=list_of_results


In [ ]:
dict_data_prediction=dict()
dict_data=dict()
dict_data["image"]="s3://cthiounn2/stage_dares/"+jpg_file
dict_data_prediction["data"]=dict_data
dict_data_prediction["predictions"]=[dict_one_prediction]

In [ ]:
import json
json.dumps(dict_data_prediction)

In [ ]:
lp.draw_box(one_image_np, layout, box_width=3, show_element_type=True).show()